**8. Calculating metrics on players level**

In the subsequent notebooks, we make an attempt to drill down further on the player level to understand how different LCBs tend to prefer different regions for making progressive passes and how much threat do they add in each region, based on their dominant foot.

The following tasks have been taken into account in this notebook:
1. Compute passing metrics (similar to match level as before) and group them by individual players

# Imports 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import re
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import itertools


C:\Users\Anuj\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [3]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

# Data 

In [4]:
# Loading clusters with VAEP values as a list of dataframes
cluster_names = ['four_defs', 'three_five_defs']
df_clusters = list()
for name in cluster_names:
    df_clusters.append(
        pd.read_pickle(
            f'D:/Storage/Datasets/data_top5/clusters/clusters_vaep/cluster_{name}.pkl'
        ))

In [5]:
# Reseting the indexes of dfs in df_clusters
for df in df_clusters:
    df.reset_index(drop=True, inplace = True)

# Creating Player-Metrics Dataset

In [6]:
def player_metrics_func(l):
    player_metrics = pd.DataFrame(columns=[
        'player_name',
        'team',
        'backline_footedness',
        'backline',
        'position',
        'wyId',
        'passes',
        'accpass',
        'accpassloc',
        'inaccpassloc',
        'accpassvaep',
        'inaccpassvaep',
        'accpassoff',
        'inaccpassoff',
        'accpassdef',
        'inaccpassdef',
        ])
    pos = [
        'RB',
        'R_CB',
        'L_CB',
        'LB',
        'RWB',
        'RCB',
        'CB',
        'LCB',
        'LWB',
        ]
    for df in l:
        for i in tqdm(range(len(df))):
            if df['backline'][i] == 4:
                s1 = list()
                for p in pos[0:4]:
                    p1 = p + '_'
                    s = [
                        [df[p][i]],
                        [df['team'][i]],
                        [df['footedness'][i]],
                        [df['backline'][i]],
                        [p],
                        [df['wyId'][i]],
                        df.loc[i,
                               list(df.columns[df.columns.str.startswith(p1)])].values,
                        ]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 3:
                s1 = list()
                for p in pos[5:8]:
                    p1 = p + '_'
                    s = [
                        [df[p][i]],
                        [df['team'][i]],
                        [df['footedness'][i]],
                        [df['backline'][i]],
                        [p],
                        [df['wyId'][i]],
                        df.loc[i,
                               list(df.columns[df.columns.str.startswith(p1)])].values,
                        ]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 5:
                s1 = list()
                for p in pos[4:]:
                    p1 = p + '_'
                    s = [
                        [df[p][i]],
                        [df['team'][i]],
                        [df['footedness'][i]],
                        [df['backline'][i]],
                        [p],
                        [df['wyId'][i]],
                        df.loc[i,
                               list(df.columns[df.columns.str.startswith(p1)])].values,
                        ]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)

    return player_metrics


In [7]:
player_metrics = player_metrics_func(df_clusters)
print(player_metrics.shape)
player_metrics.reset_index(drop=True, inplace=True)

C:\Users\Anuj\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray




(13874, 16)


In [8]:
player_metrics.tail(20)

,player_name,team,backline_footedness,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef
13854,MichelMacedoRochaMachado,Las Palmas,right-right-right-right-left,5,RWB,2565877,26,22,"[[[31.2, 12.24], [42.64, 15.64]], [[49.92, 14....","[[[32.24, 5.44], [10.4, 3.4]], [[84.24, 5.44],...","[0.0026717602740973234, 0.0018173172138631344,...","[-0.0026345248334109783, -0.014879956841468811...","[0.002675971481949091, 0.0013904524967074394, ...","[0.002500272821635008, -0.01362017635256052, -...","[-4.21120785176754e-06, 0.00042686471715569496...","[-0.005134797655045986, -0.0012597807217389345..."
13855,DavidGarciaSantana,Las Palmas,right-right-right-right-left,5,RCB,2565877,44,31,"[[[34.32, 19.04], [41.6, 17.68]], [[40.56, 13....","[[[32.24, 10.88], [72.8, 0.0]], [[47.84, 17.0]...","[0.0010353471152484417, -0.0008207135833799839...","[-0.003023515222594142, -0.003974074497818947,...","[0.001300613395869732, 1.1052470654249191e-05,...","[-0.0017481863033026457, -0.002099099569022655...","[-0.0002652662806212902, -0.000831766054034233...","[-0.0012753289192914963, -0.001874974695965647..."
13856,AlejandroGalvezJimena,Las Palmas,right-right-right-right-left,5,CB,2565877,31,28,"[[[24.96, 36.04], [32.24, 35.36]], [[29.12, 36...","[[[18.72, 36.72], [53.04, 22.44]], [[48.88, 34...","[0.0006240587681531906, 0.0032964216079562902,...","[-0.0047333454713225365, 0.00545889139175415, ...","[0.0013856561854481697, -0.0001592314802110195...","[-0.0013616452924907207, 0.0003392130602151155...","[-0.0007615974172949791, 0.0034556530881673098...","[-0.0033717004116624594, 0.005119678098708391,..."
13857,JoaquinNavarroJimenez,Las Palmas,right-right-right-right-left,5,LCB,2565877,27,20,"[[[33.28, 52.36], [24.96, 36.04]], [[33.28, 53...","[[[34.32, 50.32], [91.52, 33.32]], [[60.32, 47...","[-0.000378637807443738, 0.0009167774114757776,...","[0.0006273249164223671, -0.016335545107722282,...","[-0.00015668943524360657, 0.000356155447661876...","[0.002876903163269162, -0.015348796732723713, ...","[-0.00022194837220013142, 0.000560621963813901...","[-0.002249578246846795, -0.0009867476765066385..."
13858,DaniCastellanoBetancor,Las Palmas,right-right-right-right-left,5,LWB,2565877,11,11,"[[[40.56, 54.4], [61.36, 60.52]], [[49.92, 29....",[],"[0.00349152903072536, 0.004420250654220581, -0...",[],"[0.002971815876662731, 0.0034394124522805214, ...",[],"[0.0005197131540626287, 0.0009808380855247378,...",[]
13859,AntonioBarraganFernandez,Betis,right-right-right-left-left,5,RWB,2565893,44,32,"[[[43.68, 8.16], [49.92, 11.56]], [[70.72, 4.7...","[[[41.6, 10.2], [40.56, 20.4]], [[50.96, 3.4],...","[0.0014536645030602813, -0.004782055038958788,...","[-0.01003024447709322, -0.007644500583410263, ...","[0.001160899642854929, -0.0045260898768901825,...","[-0.002703760750591755, -0.0038482127711176872...","[0.0002927648602053523, -0.0002559651620686054...","[-0.007326483726501465, -0.003796287579461932,..."
13860,AissaMandi,Betis,right-right-right-left-left,5,RCB,2565893,85,79,"[[[26.0, 20.4], [30.16, 49.64]], [[36.4, 15.64...","[[[37.44, 14.96], [48.88, 0.0]], [[41.6, 47.6]...","[0.0007713956292718649, 0.00037431076634675264...","[-0.0061156246811151505, -0.003699769265949726...","[0.0009350981563329697, -0.0002271588891744613...","[-0.0036625410430133343, -0.001868677325546741...","[-0.00016370252706110477, 0.000601469655521214...","[-0.00245308387093246, -0.0018310918239876628,..."
13861,JordiAmatMaas,Betis,right-right-right-left-left,5,CB,2565893,75,70,"[[[34.32, 36.72], [37.44, 14.96]], [[26.0, 18....","[[[47.84, 35.36], [91.52, 32.64]], [[19.76, 21...","[0.00015649700071662664, 0.015982773154973984,...","[-0.0011390296276658773, -0.010679281316697598...","[0.0002104640007019043, 0.002512284554541111, ...","[0.0019070166163146496, -0.004950081929564476,...","[-5.396699998527765e-05, 0.013470487669110298,...","[-0.003046046243980527, -0.0057291993871331215..."
13862,HectorJuniorFirpoAd

In [9]:
# Function to add number of values of match level metrics per match
def match_idx(row):
    acc_match_idx = [len(row['accpassloc'])]
    inacc_match_idx = [len(row['inaccpassloc'])]
    return pd.Series([acc_match_idx, inacc_match_idx])

In [10]:
player_metrics[['acc_match_idx','inacc_match_idx']] = player_metrics.apply(lambda row: match_idx(row), axis=1 )

In [11]:
player_metrics.loc[player_metrics['player_name']=='AdamSmith']

,player_name,team,backline_footedness,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx
56,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499729,33,29,"[[[28.08, 4.08], [15.6, 5.44]], [[45.76, 1.36]...","[[[54.08, 13.6], [58.24, 11.56]], [[75.92, 28....","[0.0015602908097207546, 0.01085551455616951, 0...","[-0.006108248606324196, -0.01096324808895588, ...","[0.0014270162209868431, 0.006274942774325609, ...","[-0.002739163115620613, -0.009826955385506153,...","[0.00013327458873391151, 0.004580572247505188,...","[-0.003369085257872939, -0.0011362929362803698...",[29],[4]
208,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499759,56,54,"[[[33.28, 1.36], [19.76, 7.48]], [[66.56, 8.84...","[[[80.08, 8.84], [95.68, 22.44]], [[38.48, 17....","[-0.00034849834628403187, 0.003234653500840068...","[-0.003870188258588314, -0.0021765632554888725]","[-0.0004601788241416216, 0.0033038705587387085...","[-0.0027301227673888206, -0.006332803517580032]","[0.00011168047785758972, -6.921705789864063e-0...","[-0.0011400654911994934, 0.00415624026209116]",[54],[2]
276,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499772,40,35,"[[[79.04, 18.36], [101.92, 10.2]], [[91.52, 3....","[[[101.92, 1.36], [101.92, 2.04]], [[73.84, 14...","[-0.0027333241887390614, -0.005029985215514898...","[-0.029243893921375275, -0.012092892080545425,...","[-0.0030440762639045715, -0.005688074976205826...","[-0.02838980033993721, -0.01062268391251564, 0...","[0.0003107520751655102, 0.0006580897606909275,...","[-0.0008540935814380646, -0.001470208400860428...",[35],[5]
312,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499779,65,54,"[[[55.12, 8.16], [57.2, 32.64]], [[70.72, 12.9...","[[[67.6, 4.08], [81.12, 1.36]], [[30.16, 4.76]...","[0.0004147776635363698, 0.05057172104716301, 0...","[-0.009886120446026325, 0.009931659325957298, ...","[-0.00018300116062164307, 0.052904173731803894...","[-0.00873370748013258, 0.002039430197328329, -...","[0.0005977788241580129, -0.0023324524518102407...","[-0.0011524130823090672, 0.007892228662967682,...",[54],[11]
472,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499805,43,30,"[[[63.44, 1.36], [46.8, 6.8]], [[28.08, 5.44],...","[[[43.68, 1.36], [47.84, 4.08]], [[24.96, 6.12...","[-0.0027234056033194065, 0.008349435403943062,...","[-0.007041573990136385, -0.005091824568808079,...","[-0.0026750662364065647, 0.0028343996964395046...","[-0.0034799561835825443, -0.002193832071498036...","[-4.8339483328163624e-05, 0.00551503524184227,...","[-0.0035616178065538406, -0.002897992264479398...",[30],[13]
684,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499846,60,53,"[[[63.44, 11.56], [69.68, 21.76]], [[74.88, 10...","[[[27.04, 5.44], [39.52, 8.16]], [[29.12, 2.72...","[0.005353913176804781, 0.030311232432723045, 0...","[-0.005121732130646706, -0.0018177926540374756...","[0.005366073921322823, 0.030763717368245125, 0...","[-0.0014389478601515293, 0.0001220800913870334...","[-1.216074451804161e-05, -0.000452485051937401...","[-0.0036827842704951763, -0.001939872745424509...",[53],[7]
744,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499859,31,26,"[[[31.2, 8.16], [18.72, 10.88]], [[38.48, 8.84...","[[[43.68, 8.16], [62.4, 8.84]], [[31.2, 6.12],...","[0.00097347900737077, -0.0011119181290268898, ...","[-0.0016525862738490105, -0.000662904232740402...","[0.0002457159571349621, -0.00067894347012043, ...","[-0.0016500959172844887, 0.0003468878567218780...","[0.0007277630502358079, -0.0004329746589064598...","[-2.4903565645217896e-06, -0.00100979208946228...",[26],[5]
888,AdamSmith,Bournemouth,right-right-left-left,4,RB,2499884,45,39,"[[[14.56, 6.12], [9.36, 15.64]], [[48.88, 24.4...","[[[43.68, 20.4], [66.56, 1.36]], [[92.56, 3.4]...","[-0.0020024231635034084, -0.000500710681080818...","[0.001774220960214734, -0.002873945515602827, ...","[-0.0003442731685936451, -0.000260645523667335...","[-0.00013712630607187748, -0.00

In [12]:
# Function to extract dominant foot information
def extract_foot(a,b):
    pos1 = ['RB', 'R_CB', 'L_CB', 'LB']
    pos2 = ['RWB', 'RCB', 'CB', 'LCB', 'LWB']
    a = a.split('-')
    if(len(a)==4):
        return a[pos1.index(b)]
    elif(len(a)==5):
        return a[pos2.index(b)]
    elif(len(a)==3):
        return a[pos2.index(b) - 1]

In [13]:
player_metrics['footedness'] = player_metrics.apply(lambda x: extract_foot(x.backline_footedness, x.position), axis=1)
player_metrics.drop(columns = ['backline_footedness'], inplace=True)

In [14]:
player_metrics.head(5)

,player_name,team,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx,footedness
0,DannySimpson,Leicester City,4,RB,2499719,26,16,"[[[30.16, 42.84], [27.04, 65.96]], [[63.44, 24...","[[[93.6, 2.04], [92.56, 6.8]], [[27.04, 2.72],...","[-0.00042587751522660255, -0.00370064307935535...","[-0.00959702767431736, -0.005989259108901024, ...","[2.2274907678365707e-05, -0.004478777758777142...","[-0.009186144918203354, -0.0013398649170994759...","[-0.00044815242290496826, 0.000778134679421782...","[-0.00041088316356763244, -0.00464939419180154...",[16],[10],right
1,WesMorgan,Leicester City,4,R_CB,2499719,17,16,"[[[48.88, 31.96], [59.28, 33.32]], [[50.96, 59...","[[[22.88, 41.48], [55.12, 44.88]]]","[0.0034421561285853386, -0.0025732177309691906...",[0.007936863228678703],"[0.0028925221413373947, -0.0033792490139603615...",[0.0004197852686047554],"[0.0005496339872479439, 0.0008060311665758491,...",[0.007517078425735235],[16],[1],right
2,HarryMaguire,Leicester City,4,L_CB,2499719,23,22,"[[[23.92, 51.0], [40.56, 57.8]], [[24.96, 46.9...","[[[39.52, 39.44], [76.96, 57.12]]]","[0.007495604921132326, 0.008959720842540264, 0...",[-0.004673151299357414],"[0.0022297450341284275, 0.0036585405468940735,...",[-0.0009126844815909863],"[0.005265859887003899, 0.005301180295646191, 0...",[-0.003760466817766428],[22],[1],right
3,ChristianFuchs,Leicester City,4,LB,2499719,32,20,"[[[95.68, 61.2], [96.72, 42.84]], [[94.64, 63....","[[[23.92, 64.6], [65.52, 55.08]], [[88.4, 63.9...","[0.07473685592412949, 0.03496371582150459, 0.0...","[-0.002852288307622075, -0.004100826568901539,...","[0.074192576110363, 0.035530250519514084, 0.00...","[-4.0952349081635475e-05, -0.00330827571451663...","[0.0005442826077342033, -0.000566533301025629,...","[-0.0028113359585404396, -0.000792550621554255...",[20],[12],left
4,BrunoSaltorGrau,Brighton,4,RB,2499720,25,17,"[[[65.52, 5.44], [65.52, 17.0]], [[88.4, 4.76]...","[[[34.32, 13.6], [55.12, 34.68]], [[34.32, 21....","[-0.0019361847080290318, 0.02846207097172737, ...","[-0.007493174634873867, -0.006757671479135752,...","[-0.0019506150856614113, 0.029143858700990677,...","[-0.0037297995295375586, -0.005478677339851856...","[1.4430377632379532e-05, -0.000681788078509271...","[-0.003763374872505665, -0.0012789941392838955...",[17],[8],right


In [15]:
# Import team contribution data
team_contri = pd.read_pickle("D:/Storage/Datasets/data_top5/vaep/value_sums.pkl")

In [16]:
# Merge team contribution data with player metrics
player_metrics_contri = pd.merge(player_metrics,
                                 team_contri,
                                 left_on=['wyId', 'team'],
                                 right_on=['match_id', 'team_name'])

In [17]:
player_metrics_contri.shape

(13874, 22)

In [18]:
player_metrics_contri.drop(['team_name','match_id'],axis=1,inplace=True)

In [19]:
# Group the data by individual players, their team, role and dominant foot
df = player_metrics_contri.groupby([
    'player_name', 'team', 'position', 'footedness'
])['passes', 'accpass', 'accpassloc', 'inaccpassloc', 'accpassvaep',
   'inaccpassvaep', 'accpassoff', 'inaccpassoff', 'accpassdef', 'inaccpassdef',
    'acc_match_idx', 'inacc_match_idx', 'offsum_regionwise',
   'vaepsum_regionwise'].sum().reset_index()

In [20]:
player_metrics_contri['matches_played'] = pd.Series(
    np.ones(len(player_metrics_contri)))
df1 = player_metrics_contri.groupby(
    ['player_name', 'team', 'position',
     'footedness'])['matches_played'].sum().reset_index()
df['matches_played'] = df1.matches_played

In [21]:
df.head()

,player_name,team,position,footedness,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx,offsum_regionwise,vaepsum_regionwise,matches_played
0,AaronCresswell,West Ham,LB,left,275.0,203.0,"[[[55.12, 65.28], [34.32, 22.44]], [[54.08, 61...","[[[30.16, 66.64], [70.72, 48.96]], [[60.32, 62...","[0.0011366972466930747, -0.005854686256498098,...","[-0.0025188359431922436, -0.00414131348952651,...","[0.001508316956460476, -0.005413928534835577, ...","[0.0003642616793513298, -0.0028084642253816128...","[-0.0003716197097674012, -0.000440757721662521...","[-0.0028830976225435734, -0.001332849380560219...","[34, 38, 17, 30, 31, 38, 15]","[7, 12, 10, 4, 13, 17, 9]","[-0.07621848455164582, 0.2054318905575201, 0.2...","[-0.16642074729315937, 0.15280095604248345, 0....",7.0
1,AaronCresswell,West Ham,LCB,left,796.0,644.0,"[[[33.28, 51.0], [79.04, 40.8]], [[28.08, 51.0...","[[[44.72, 45.56], [102.96, 6.8]], [[93.6, 59.8...","[0.021589750424027443, 0.020117048174142838, 0...","[-0.007777335122227669, -0.012902607209980488,...","[0.022047633305191994, 0.0012021067086607218, ...","[-0.006338434759527445, -0.012673381716012955,...","[-0.00045788311399519444, 0.01891494169831276,...","[-0.0014389004791155457, -0.000229225726798176...","[38, 13, 24, 34, 38, 23, 41, 17, 26, 17, 20, 4...","[10, 2, 7, 7, 11, 7, 12, 7, 3, 7, 4, 7, 13, 5,...","[0.27694848889950663, 0.28493986767716706, 0.3...","[0.25020204787142575, 0.2746145740384236, 0.31...",20.0
2,AaronMartinCaricol,Espanyol,LB,left,1044.0,813.0,"[[[73.84, 66.64], [66.56, 60.52]], [[60.32, 62...","[[[45.76, 67.32], [58.24, 68.0]], [[74.88, 65....","[0.0026829878333956003, -0.0031049377284944057...","[-0.004867118317633867, -0.003997172694653273,...","[0.0026981597766280174, -0.003319632727652788,...","[-0.002615994308143854, -0.003058362752199173,...","[-1.5171943232417107e-05, 0.000214695115573704...","[-0.002251124009490013, -0.0009388098260387778...","[22, 39, 9, 30, 19, 22, 31, 37, 22, 24, 35, 37...","[8, 8, 4, 8, 6, 1, 6, 10, 7, 8, 6, 11, 11, 4, ...","[0.9028447055025026, 0.31244528922252357, 0.14...","[0.8969237081473693, 0.2685900873038918, 0.117...",31.0
3,AaronWanBissaka,Crystal Palace,RB,right,226.0,165.0,"[[[26.0, 6.12], [28.08, 29.24]], [[34.32, 3.4]...","[[[22.88, 4.76], [54.08, 18.36]], [[14.56, 6.1...","[0.004955056123435497, 0.0056464336812496185, ...","[-0.004906652960926294, -0.0042328257113695145...","[0.001684062648564577, 0.005129782948642969, 0...","[-0.0005502565763890743, -0.000924525549635291...","[0.0032709932420402765, 0.000516650965437293, ...","[-0.00435639638453722, -0.0033082999289035797,...","[5, 15, 26, 26, 10, 30, 53]","[11, 6, 9, 15, 10, 5, 5]","[0.1547394865192473, 0.11134780140127987, 0.15...","[0.22213642322458327, 0.10213371750432998, 0.1...",7.0
4,AbdallahN'Dour,Strasbourg,LB,left,84.0,68.0,"[[[27.04, 55.08], [39.52, 51.0]], [[67.6, 51.6...","[[[91.52, 64.6], [104.0, 0.0]], [[79.04, 62.56...","[0.01744888722896576, 0.009277098812162876, 0....","[-0.004355764016509056, -0.003133192425593734,...","[0.002800592454150319, 0.008430924266576767, 0...","[-0.004175558686256409, -0.003061573952436447,...","[0.01464829407632351, 0.0008461743127554655, 0...","[-0.00018020509742200375, -7.161847315728664e-...","[19, 15, 34]","[6, 5, 5]","[0.5784799118991941, 0.4648994808085263, 0.344...","[0.43872065003961325, 0.407610188703984, 0.326...",3.0


In [22]:
df.shape

(1905, 19)

In [24]:
df.to_pickle('D:/Storage/Datasets/data_top5/defender_clusters/defender_clustering_dataset.pkl')

In [4]:
df = pd.read_pickle('D:/Storage/Datasets/data_top5/defender_clusters/defender_clustering_dataset.pkl')
df[df['player_name'] == 'SokratisPapastathopoulos']

,player_name,team,position,footedness,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx,offsum_regionwise,vaepsum_regionwise,matches_played
1706,SokratisPapastathopoulos,Dortmund,CB,right,44.0,41.0,"[[[30.16, 36.04], [28.08, 10.2]], [[27.04, 35....","[[[8.32, 7.48], [55.12, 31.96]], [[35.36, 36.0...","[2.6425812393426895e-05, 0.002308398950845003,...","[0.005252080038189888, -0.0038699712604284286,...","[3.4714583307504654e-05, 0.0015599741600453854...","[0.0006327704759314656, -0.0036619026213884354...","[-8.288770914077759e-06, 0.0007484247907996178...","[0.004619309678673744, -0.00020806863903999329...",[41],[3],"[0.03064593207091093, 0.34969412034843117, 0.2...","[0.021246155258268118, 0.28814812656491995, 0....",1.0
1707,SokratisPapastathopoulos,Dortmund,L_CB,right,150.0,146.0,"[[[42.64, 20.4], [45.76, 6.8]], [[38.48, 20.4]...","[[[46.8, 18.36], [88.4, 16.32]], [[26.0, 52.36...","[-0.003242572769522667, 0.0011798432096838951,...","[-0.0025326493196189404, -0.004535330925136805...","[-0.0029240953736007214, 0.0003842860460281372...","[-0.0012859636917710304, -0.001700182445347309...","[-0.0003184775123372674, 0.0007955571636557579...","[-0.001246685511432588, -0.0028351484797894955...",[146],[4],"[-1.1166958776302636, 0.9779443047009408, 1.25...","[-1.1808214032789692, 0.9298764802515507, 1.22...",1.0
1708,SokratisPapastathopoulos,Dortmund,RCB,right,162.0,149.0,"[[[31.2, 9.52], [45.76, 4.08]], [[35.36, 14.96...","[[[11.44, 34.0], [64.48, 0.0]], [[72.8, 17.0],...","[0.001147298258729279, -0.00035851215943694115...","[0.007024459540843964, -0.013801105320453644, ...","[0.0007502725347876549, -0.0004894016310572624...","[-0.0009486133931204677, -0.011470727622509003...","[0.00039702572394162416, 0.0001308894716203212...","[0.007973073050379753, -0.0023303779307752848,...","[77, 72]","[2, 11]","[0.5606257404433563, 0.17548426799476147, 0.57...","[0.5660167490132153, 0.19014312722720206, 0.53...",2.0
1709,SokratisPapastathopoulos,Dortmund,R_CB,right,1388.0,1235.0,"[[[31.2, 29.92], [35.36, 47.6]], [[54.08, 22.4...","[[[54.08, 17.68], [87.36, 59.84]], [[13.52, 17...","[0.0013178364606574178, -0.0004410729743540287...","[-0.0033364498522132635, -0.003568395739421248...","[0.0008247480727732182, -0.0004229787737131119...","[-0.0012069004587829113, -4.25062607973814e-05...","[0.0004930883878841996, -1.8094200640916824e-0...","[-0.002129549393430352, -0.003525889478623867,...","[94, 44, 104, 20, 143, 39, 30, 28, 13, 7, 67, ...","[9, 5, 8, 5, 8, 13, 6, 10, 0, 2, 6, 6, 5, 8, 1...","[2.1290984835941344, 0.7319148674141616, 0.914...","[2.0870959074236453, 0.7658658343134448, 0.899...",24.0
